In [1]:
import os.path as osp
from PostProcess import *

In [2]:
video = 'CB(C)20230531_10fps'

In [3]:
index_label = {
    0: 'outside',
    1: 'nonsense',
    2: 'ileocecal',
    3: 'bbps0',
    4: 'bbps1',
    5: 'bbps2',
    6: 'bbps3',
}

In [4]:
do_fix_outside = True
do_fix_nonsense = True

In [5]:
exp_root = '/mnt/data/cwy/ColonoscopyQualityControl/Experiment/R10001_detect_fps_vitp14s336c7'
#exp_root = '/mnt/data/cwy/ColonoscopyQualityControl/Experiment/SingleTask_VGG19'

In [6]:
pred_json_root = osp.join(exp_root, 'predicts', video)
pred_json_path = osp.join(pred_json_root, 'predict_result.json')
#pred_json_root = '/mnt/data/cwy/ColonoscopyQualityControl/Experiment/SingleTask_VGG19'
#pred_json_path = '/mnt/data/cwy/ColonoscopyQualityControl/Experiment/SingleTask_VGG19/pred_merged_result.json'

In [7]:
outlier_json_root = osp.join(exp_root, 'outliers', video)
outlier_json_path = osp.join(outlier_json_root,'outlier_result.json')
#outlier_json_root = '/mnt/data/cwy/ColonoscopyQualityControl/Experiment/R1001_detect_SingleTask_VGG19/outliers/SingleTask_VGG19'
#outlier_json_path = '/mnt/data/cwy/ColonoscopyQualityControl/Experiment/R1001_detect_SingleTask_VGG19/outliers/SingleTask_VGG19/outlier_result.json'

In [8]:
d = json.load(open(pred_json_path, 'r'))
with open(osp.join(pred_json_root, 'pred.txt'),'w') as f:
    for k, v in d.items():
        print(f"{video}/"+k,file=f,end=' ')
        res = []
        if v[0] == 1: res.append(4)
        if v[1] == 1: res.append(5)
        if v[2] == 1: res.append(6)
        if v[3] == 1: res.append(0)
        if v[4] == 1: res.append(1)
        if v[5] == 1: res.append(2)
        if v[6] == 1: res.append(3)
        sorted(res,reverse=False)
        for it in res:
            print(it,file=f,end=' ')
        f.write('\n')

In [9]:
gt_path = osp.join(pred_json_root, 'gt.txt')
pred_path = osp.join(pred_json_root, 'pred.txt')

In [10]:
def read_from_imagenet(txt_path: str):
    name_all = []
    label_all = []
    with open(txt_path, 'r') as f:
        lines = f.readlines()
    for line in lines:
        # 0位置是图片名
        line = line.strip()
        name = line.split(' ')[0]
        name_all.append(name)
        try:
            labels = [int(x)-1 for x in line.split(' ')[1:]]
        except ValueError:
            print(line)
            raise
        label_list = [0, 0, 0, 0, 0, 0, 0]
        for label in labels:
            label_list[label] = 1
        # 把后面4个bbps标签移动到最前面
        label_list = label_list[3:] + label_list[:3]
        label_all.append(label_list)

    return name_all, torch.tensor(label_all)

In [11]:
pred_names, pred_labels = read_from_imagenet(pred_path)
# gt_labels与pred_labels的每一行是一个长度为7的标签向量，分别对应label_dict的每个类别

In [12]:
pred_ori = pred_labels.clone().detach()

In [13]:
pred_fix = post_process(pred_labels, outlier_json_path, do_fix_outside, do_fix_nonsense)

Fix [367, 367]: 367=[0, 1, 0, 0, 0, 0, 0]
Fix [592, 593]: 592=[0, 1, 0, 0, 0, 0, 0]
Fix [592, 593]: 593=[0, 1, 0, 0, 0, 0, 0]
Fix [1465, 1466]: 1465=[0, 1, 0, 0, 0, 0, 0]
Fix [1465, 1466]: 1466=[0, 1, 0, 0, 0, 0, 0]
Fix [2042, 2042]: 2042=[0, 1, 0, 0, 0, 0, 0]
Fix [2196, 2196]: 2196=[0, 1, 0, 0, 0, 0, 0]
Fix [3795, 3795]: 3795=[0, 1, 0, 0, 0, 0, 0]
Fix [3849, 3856]: 3849=[0, 1, 0, 0, 0, 0, 0]
Fix [3849, 3856]: 3850=[0, 1, 0, 0, 0, 0, 0]
Fix [3849, 3856]: 3851 not change
Fix [3849, 3856]: 3852 not change
Fix [3849, 3856]: 3853 not change
Fix [3849, 3856]: 3854 not change
Fix [3849, 3856]: 3855 not change
Fix [3849, 3856]: 3856=[0, 1, 0, 0, 0, 0, 0]
Fix [5009, 5009]: 5009=[0, 1, 0, 0, 0, 0, 0]
Fix [5456, 5502]: 5456=[0, 1, 0, 0, 0, 0, 0]
Fix [5456, 5502]: 5457 not change
Fix [5456, 5502]: 5458=[0, 1, 0, 0, 0, 0, 0]
Fix [5456, 5502]: 5459 not change
Fix [5456, 5502]: 5460=[0, 1, 0, 0, 0, 0, 0]
Fix [5456, 5502]: 5461=[0, 1, 0, 0, 0, 0, 0]
Fix [5456, 5502]: 5462=[0, 1, 0, 0, 0, 0, 0]
Fix [5

/mnt/data/cwy/ColonoscopyQualityControl/PostProcess.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred_fix[now_i] = torch.tensor(val_func(pred_origin, item, now_i))


In [14]:
pred_labels = pred_fix

# 评估

In [15]:
gt_names, gt_labels = read_from_imagenet(gt_path)
# gt_labels与pred_labels的每一行是一个长度为7的标签向量，分别对应label_dict的每个类别

In [16]:
confuse_matrix = dict()
for i in range(3):
    confuse_matrix[f'label_{index_label[i]}_TP'] = 0.
    confuse_matrix[f'label_{index_label[i]}_FP'] = 0.
    confuse_matrix[f'label_{index_label[i]}_FN'] = 0.
    confuse_matrix[f'label_{index_label[i]}_TN'] = 0.
for i in range(0, 4):  # i: predict
    for j in range(0, 4):  # j: gt
        confuse_matrix[f'label_cleansing_pred_{index_label[i + 3]}_gt_{index_label[j + 3]}'] = 0.

In [17]:
# 计算test_acc
mean_acc = float(torch.eq(pred_labels, gt_labels).float().mean().cpu())

In [18]:
in_out_labels = pred_labels[:, 0]
# 体内外标签: BoolTensor[B]
# outside时为True
label_in_out_pred = torch.gt(in_out_labels, 0)
# 体内外gt: BoolTensor[B]
label_in_out_gt = torch.gt(gt_labels[:, 0], 0)
confuse_matrix[f'label_{index_label[0]}_TP'] += float((label_in_out_pred & label_in_out_gt).float().sum().cpu())
confuse_matrix[f'label_{index_label[0]}_FP'] += float((label_in_out_pred & ~label_in_out_gt).float().sum().cpu())
confuse_matrix[f'label_{index_label[0]}_FN'] += float((~label_in_out_pred & label_in_out_gt).float().sum().cpu())
confuse_matrix[f'label_{index_label[0]}_TN'] += float((~label_in_out_pred & ~label_in_out_gt).float().sum().cpu())


In [19]:
nonsense_logit = pred_labels[:, 1]
# 坏帧标签: BoolTensor[B]
# nonsense时为True
label_nonsense_pred = torch.gt(nonsense_logit, 0)
# 帧质量gt: BoolTensor[B]
# pred或gt是outside时不计入总数
label_nonsense_gt = torch.gt(gt_labels[:, 1], 0)
flag = ~label_in_out_pred & ~label_in_out_gt
confuse_matrix[f'label_{index_label[1]}_TP'] += float(
    (flag & label_nonsense_pred & label_nonsense_gt).float().sum().cpu())
confuse_matrix[f'label_{index_label[1]}_FP'] += float(
    (flag & label_nonsense_pred & ~label_nonsense_gt).float().sum().cpu())
confuse_matrix[f'label_{index_label[1]}_FN'] += float(
    (flag & ~label_nonsense_pred & label_nonsense_gt).float().sum().cpu())
confuse_matrix[f'label_{index_label[1]}_TN'] += float(
    (flag & ~label_nonsense_pred & ~label_nonsense_gt).float().sum().cpu())


In [20]:
ileo_logit = pred_labels[:, 2]
# 回盲部标签: BoolTensor[B]
label_ileo_pred = torch.gt(ileo_logit, 0)
# 回盲部gt: BoolTensor[B]
label_ileo_gt = torch.gt(gt_labels[:, 2], 0)
flag = ~label_in_out_pred & ~label_in_out_gt & ~label_nonsense_pred & ~label_nonsense_gt
confuse_matrix[f'label_{index_label[2]}_TP'] += float((flag & label_ileo_pred & label_ileo_gt).float().sum().cpu())
confuse_matrix[f'label_{index_label[2]}_FP'] += float((flag & label_ileo_pred & ~label_ileo_gt).float().sum().cpu())
confuse_matrix[f'label_{index_label[2]}_FN'] += float((flag & ~label_ileo_pred & label_ileo_gt).float().sum().cpu())
confuse_matrix[f'label_{index_label[2]}_TN'] += float((flag & ~label_ileo_pred & ~label_ileo_gt).float().sum().cpu())

In [21]:
# 清洁度logit: FloatTensor[B, 4]
cls_logit = pred_labels[:, 3:]
# 清洁度label: IntTensor[B] (取预测值最大的，但会被outside标签抑制)
label_cls_pred = torch.argmax(cls_logit, dim=-1)
# 清洁度gt: IntTensor[B]
label_cls_gt = torch.argmax(gt_labels[:, 3:], dim=-1)
flag = ~label_in_out_pred & ~label_in_out_gt & ~label_nonsense_pred & ~label_nonsense_gt
for i in range(0, 4):  # i: predict
    for j in range(0, 4):  # j: gt
        confuse_matrix[f'label_cleansing_pred_{index_label[i + 3]}_gt_{index_label[j + 3]}'] += float((flag & torch.eq(label_cls_pred, i) & torch.eq(label_cls_gt, j)).float().sum().cpu())  # flag用于清洁度标签抑制

In [22]:
confuse_matrix

{'label_outside_TP': 535.0,
 'label_outside_FP': 0.0,
 'label_outside_FN': 56.0,
 'label_outside_TN': 5227.0,
 'label_nonsense_TP': 1975.0,
 'label_nonsense_FP': 41.0,
 'label_nonsense_FN': 151.0,
 'label_nonsense_TN': 3060.0,
 'label_ileocecal_TP': 108.0,
 'label_ileocecal_FP': 6.0,
 'label_ileocecal_FN': 36.0,
 'label_ileocecal_TN': 2910.0,
 'label_cleansing_pred_bbps0_gt_bbps0': 27.0,
 'label_cleansing_pred_bbps0_gt_bbps1': 7.0,
 'label_cleansing_pred_bbps0_gt_bbps2': 0.0,
 'label_cleansing_pred_bbps0_gt_bbps3': 0.0,
 'label_cleansing_pred_bbps1_gt_bbps0': 14.0,
 'label_cleansing_pred_bbps1_gt_bbps1': 203.0,
 'label_cleansing_pred_bbps1_gt_bbps2': 10.0,
 'label_cleansing_pred_bbps1_gt_bbps3': 0.0,
 'label_cleansing_pred_bbps2_gt_bbps0': 0.0,
 'label_cleansing_pred_bbps2_gt_bbps1': 70.0,
 'label_cleansing_pred_bbps2_gt_bbps2': 856.0,
 'label_cleansing_pred_bbps2_gt_bbps3': 75.0,
 'label_cleansing_pred_bbps3_gt_bbps0': 0.0,
 'label_cleansing_pred_bbps3_gt_bbps1': 3.0,
 'label_cleansin

In [23]:
metrics = dict()

# 体内外
TP: float = confuse_matrix[f'label_{index_label[0]}_TP']
FP: float = confuse_matrix[f'label_{index_label[0]}_FP']
FN: float = confuse_matrix[f'label_{index_label[0]}_FN']
TN: float = confuse_matrix[f'label_{index_label[0]}_TN']
metrics[f'label_{index_label[0]}_acc'] = (TP + TN) / (TP + FP + FN + TN) if TP + FP + FN + TN > 0. else 0.

# 帧质量
TP: float = confuse_matrix[f'label_{index_label[1]}_TP']
FP: float = confuse_matrix[f'label_{index_label[1]}_FP']
FN: float = confuse_matrix[f'label_{index_label[1]}_FN']
TN: float = confuse_matrix[f'label_{index_label[1]}_TN']
metrics[f'label_{index_label[1]}_acc'] = (TP + TN) / (TP + FP + FN + TN) if TP + FP + FN + TN > 0. else 0.

# 回盲部
TP: float = confuse_matrix[f'label_{index_label[2]}_TP']
FP: float = confuse_matrix[f'label_{index_label[2]}_FP']
FN: float = confuse_matrix[f'label_{index_label[2]}_FN']
TN: float = confuse_matrix[f'label_{index_label[2]}_TN']

metrics[f'label_{index_label[2]}_acc'] = (TP + TN) / (TP + FP + FN + TN) if TP + FP + FN + TN > 0. else 0.

# 四分清洁度准确率
total: float = 0.
correct: float = 0.
for i in range(3, 7):  # i: predict
    for j in range(3, 7):  # j: gt
        tmp = confuse_matrix[f'label_cleansing_pred_{index_label[i]}_gt_{index_label[j]}']
        if i == j:
            correct += tmp
        total += tmp
metrics[f'label_cleansing_acc'] = correct / total if total > 0. else 0.

In [24]:
mean_acc

0.973284900188446

In [25]:
metrics

{'label_outside_acc': 0.9903746992093503,
 'label_nonsense_acc': 0.9632676487468912,
 'label_ileocecal_acc': 0.9862745098039216,
 'label_cleansing_acc': 0.9101307189542484}

In [77]:
for i in range(pred_labels.size()[0]):
    if torch.not_equal(pred_labels[i],gt_labels[i]).any():
        print(f"{i}:{pred_labels[i].numpy()} != {gt_labels[i].numpy()}")

# 可视化

In [80]:
import DrawLabelColorBlock
import os
import os.path as osp
import shutil

In [81]:
frames_pred_root = osp.join(exp_root, 'frames_with_pred')
os.makedirs(frames_pred_root, exist_ok=True)
frames_ori_path = osp.join(exp_root, 'frames', video)
frames_pred_path = osp.join(frames_pred_root, video)
if osp.exists(frames_pred_path):
    shutil.rmtree(frames_pred_path)
os.makedirs(frames_pred_path)

In [82]:
for i,pd in enumerate(pred_labels):
    ori_img_path = osp.join(frames_ori_path, osp.split(pred_names[i])[-1])
    out_img_path = osp.join(frames_pred_path,osp.split(pred_names[i])[-1])
    pd_np=pd.numpy()
    pd_dict={'outside': False,
           'nonsense': False,
           'ileocecal': False,
           'bbps': -1}
    if pd_np[0]==1:
        pd_dict['outside']=True
    if pd_np[1]==1:
        pd_dict['nonsense']=True
    if pd_np[2]==1:
        pd_dict['ileocecal']=True
    if pd_np[3]==1:
        pd_dict['bbps']=0
    elif pd_np[4]==1:
        pd_dict['bbps']=1
    elif pd_np[5]==1:
        pd_dict['bbps']=2
    elif pd_np[6]==1:
        pd_dict['bbps']=3
    DrawLabelColorBlock.DrawLabelColorBlockOnFrame(ori_img_path,out_img_path,**pd_dict)

In [83]:
# 把帧拼成视频
img_list = os.listdir(frames_pred_path)
img_list.sort(key=lambda x: int(x.split('.')[0]))
img_list

['000000.png',
 '000002.png',
 '000004.png',
 '000006.png',
 '000008.png',
 '000010.png',
 '000012.png',
 '000014.png',
 '000016.png',
 '000018.png',
 '000020.png',
 '000022.png',
 '000024.png',
 '000026.png',
 '000028.png',
 '000030.png',
 '000032.png',
 '000034.png',
 '000036.png',
 '000038.png',
 '000040.png',
 '000042.png',
 '000044.png',
 '000046.png',
 '000048.png',
 '000050.png',
 '000052.png',
 '000054.png',
 '000056.png',
 '000058.png',
 '000060.png',
 '000062.png',
 '000064.png',
 '000066.png',
 '000068.png',
 '000070.png',
 '000072.png',
 '000074.png',
 '000076.png',
 '000078.png',
 '000080.png',
 '000082.png',
 '000084.png',
 '000086.png',
 '000088.png',
 '000090.png',
 '000092.png',
 '000094.png',
 '000096.png',
 '000098.png',
 '000100.png',
 '000102.png',
 '000104.png',
 '000106.png',
 '000108.png',
 '000110.png',
 '000112.png',
 '000114.png',
 '000116.png',
 '000118.png',
 '000120.png',
 '000122.png',
 '000124.png',
 '000126.png',
 '000128.png',
 '000130.png',
 '000132.p

In [84]:
video_pred_root = osp.join(exp_root, 'videos_with_pred')
os.makedirs(video_pred_root, exist_ok=True)

In [85]:
import cv2
# cv2
img = cv2.imread(osp.join(frames_pred_path, img_list[0]))
height, width, layers = img.shape

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter(osp.join(video_pred_root, f'{video}.mp4'), fourcc, 12.5, (width, height))

for img_name in img_list:
    img = cv2.imread(osp.join(frames_pred_path, img_name))
    video.write(img)
    
cv2.destroyAllWindows()
video.release()